In [ ]:
# 第八章作业
# 作业1： 使用 GPTQ 量化 OPT-6.7B 模型
import os

os.environ['HF_HOME'] = 'D:\MTIDE\.cache\huggingface'
os.environ['HF_HUB_CACHE'] = 'D:\MTIDE\.cache\huggingface\hub'

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

# 使用预先从 HuggingFace 下载的模型文件
# model_name_or_path = "facebook\opt-6.7b"
model_name_or_path = "D:\MTIDE\code\AI\models\facebook\opt-6.7b"
model_dir = 'models/opt-6.7b-gptq'

# 使用 GPTQ 算法支持的默认数据集来量化
quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

In [ ]:
# 保存模型权重
quant_model.save_pretrained(model_dir)

In [ ]:
#使用 GPU 加载模型并生成文本

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
# 使用自定义数据集量化模型

from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))